This notebook highlights the inequalities between different regions in terms of access to a water supply service.

CDP data are combined with the Water Poverty Index (WPI) to show areas where drinking water issues are critical.

The WPI is an index that takes into account water resources, access to water, water use and the environment. See more [here](http://ihp-wins.unesco.org/)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px

Cities_disclosing_df = pd.read_csv("../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv")
Cities_disclosing_df.drop(['Year Reported to CDP','Organization', 'Reporting Authority', 'Access', 'First Time Discloser', 'Population Year','Last update'], axis = 1, inplace = True)

MPI = pd.read_csv('../input/water-poverty-index-wpi/Water poverty index.csv') 
MPI.drop(['the_geom','FID','OBJECTID','ISO','UNESCOMEMB'], axis = 1, inplace = True)
MPI.columns = ['Country', 'WPI']
MPI['Country'] =MPI['Country'].replace({"Cote d'Ivoire":"Côte d'Ivoire"})
MPI['Country'] =MPI['Country'].replace({"Palestine":"State of Palestine"})
MPI.loc[MPI.shape[0], 'Country'] = 'Taiwan, Greater China'
MPI.loc[MPI.shape[0]-1, 'WPI'] = 56.6
MPI.loc[MPI.shape[0], 'Country'] = 'Gibraltar'
MPI.loc[MPI.shape[0]-1, 'WPI'] = -999.0
MPI.loc[MPI.shape[0], 'Country'] = 'China, Hong Kong Special Administrative Region'
MPI.loc[MPI.shape[0]-1, 'WPI'] = 56.6

merge_df = pd.merge(Cities_disclosing_df, MPI)
merge_df['Population'][548]=1383432134.0/1000
merge_df['Population'][396]=399724000.0/100
merge_df.sort_values(by = 'Population', inplace = True)

cities_df = pd.read_csv("../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")
cities_df['Response Answer'] = cities_df['Response Answer'].fillna('No Response')

def convert_float(x):
    try :
        x = float(x)
        return x
    except :
        return x
    
Q141 = cities_df[cities_df['Question Number'] == '14.1']
Q141['Response Answer'] = Q141['Response Answer'].apply(convert_float)
Q141.drop(['Questionnaire', 'Year Reported to CDP', 'CDP Region', 'Parent Section', 'Section','Question Number', 'Question Name', 'Column Number', 'Column Name','Row Number', 'Row Name', 'Comments', 'File Name','Last update'], axis = 1, inplace = True)
# Replace response with errors
Q141['Response Answer'][857449]=95.52
Q141['Response Answer'][246477]=96.
Q141.drop(Q141[Q141['Response Answer'] == 'No Response'].index, inplace = True)

Merge_q141 = pd.merge(Q141, merge_df)
Merge_q141 = Merge_q141.drop(Merge_q141[Merge_q141['Response Answer'] == 'No Response'].index)
Merge_q141['Response Answer'].astype(np.float)
Merge_q141.sort_values(by = 'Response Answer', inplace = True)

df = Merge_q141[Merge_q141['WPI'] != -999.0]
df = df[df['Response Answer'] != 'No Response']

In [ ]:
df.sort_values(by = 'CDP Region', inplace = True)
df['CDP Region']=pd.Categorical(df['CDP Region'])
 
# create data
x = df['Response Answer']
y = df['WPI']
z = df['Population']/5000

# plot
plt.rcParams["legend.title_fontsize"] = "large"
plt.rcParams["font.size"] = 20
plt.rcParams["font.sans-serif"] = 'Arial'

plt.figure(figsize=(22, 10))
plt.scatter(x, y, s=z, c=df['CDP Region'].cat.codes, cmap=plt.cm.get_cmap('RdYlBu', 8), alpha=0.6, edgecolors="grey", linewidth=2)

# Add titles (main and on axis)
plt.xlabel("City's population with a water supply service (%)")
plt.ylabel("Water poverty index (WPI) of the city's country")
plt.title("Inequalities of regions regarding access to water supply service")


# This function formatter will replace integers with target names
formatter = plt.FuncFormatter(lambda val, loc: df['CDP Region'].unique()[val])

# We must be sure to specify the ticks matching our target names
plt.colorbar(ticks=[0, 1, 2, 3, 4, 5, 6, 7], format=formatter);

# Set the clim so that labels are centered on each block
plt.clim(-0.5, 7.5)


for area in [750, 1500, 3000]:
    plt.scatter([], [], c='k', alpha=0.4, s=area,label='     '+str(area/3000*2)+ ' M')
plt.legend(scatterpoints=1, frameon=False, labelspacing=2,loc='lower left',bbox_to_anchor=(0.05, 0.1, 0.5, 0.5), title='Population')

The region with the greatest lack of overall water supply service is Africa.

Are the cities most concerned taking appropriate measures to improve their water supply service?

In [ ]:
Q143 = cities_df[cities_df['Question Number'] == '14.3']
Q143w = Q143[Q143['Response Answer']=='Inadequate or ageing water supply infrastructure']

Q143_actions = pd.DataFrame()

for index in Q143w.index:
    Row_Number = Q143['Row Number'][index]
    Account_Number = Q143['Account Number'][index]
    df1 = Q143[Q143['Account Number']==Account_Number]
    df1 = df1[df1['Row Number']==Row_Number]
    Q143_actions = pd.concat([Q143_actions,df1])
    
Q143_actions = Q143_actions[Q143_actions['Response Answer']=='Investment in existing water supply infrastructure']

Q143_status_action = pd.DataFrame()
for index in Q143_actions.index:
    Row_Number = Q143['Row Number'][index]
    Account_Number = Q143['Account Number'][index]
    df2 = Q143[Q143['Account Number']==Account_Number]
    df2 = df2[df2['Row Number']==Row_Number]
    Q143_status_action = pd.concat([Q143_status_action,df2])
    
Q143_status_action1 = Q143_status_action[Q143_status_action['Response Answer']=='Scoping']
Q143_status_action2 = Q143_status_action[Q143_status_action['Response Answer']=='Pre-feasibility study/impact assessment']
Q143_status_action3 = Q143_status_action[Q143_status_action['Response Answer']=='Pre-implementation']
Q143_status_action4 = Q143_status_action[Q143_status_action['Response Answer']=='Implementation']
Q143_status_action5 = Q143_status_action[Q143_status_action['Response Answer']=='Implementation complete but not in operation']
Q143_status_action6 = Q143_status_action[Q143_status_action['Response Answer']=='Operation']
Q143_status_action7 = Q143_status_action[Q143_status_action['Response Answer']=='Monitoring and reporting']

merge_df['Risks'] = 'Not identified as a risk'
for index in merge_df.index:
    for j in Q143w.index:
        Account_Number = Q143w['Account Number'][j]
        if merge_df['Account Number'][index]==Account_Number:
            for k in Q143_actions.index :
                action = Q143_actions['Account Number'][k]
                if merge_df['Account Number'][index]==action:
                    merge_df['Risks'][index]='No investment in existing water supply infrastructure'
                    for h in Q143_status_action1.index:
                        status =Q143_status_action1['Account Number'][h]
                        if merge_df['Account Number'][index]==status: 
                            merge_df['Risks'][index]='Scoping'
                    for h in Q143_status_action2.index:
                        status =Q143_status_action2['Account Number'][h]
                        if merge_df['Account Number'][index]==status: 
                            merge_df['Risks'][index]='Pre-feasibility study/impact assessment'
                    for h in Q143_status_action3.index:
                        status =Q143_status_action3['Account Number'][h]
                        if merge_df['Account Number'][index]==status:
                            merge_df['Risks'][index]='Pre-implementation'
                    for h in Q143_status_action4.index:
                        status =Q143_status_action4['Account Number'][h]
                        if merge_df['Account Number'][index]==status:
                            merge_df['Risks'][index]='Implementation'
                    for h in Q143_status_action5.index:
                        status =Q143_status_action5['Account Number'][h]
                        if merge_df['Account Number'][index]==status:
                            merge_df['Risks'][index]='Implementation complete but not in operation'
                    for h in Q143_status_action6.index:
                        status =Q143_status_action6['Account Number'][h]
                        if merge_df['Account Number'][index]==status:
                            merge_df['Risks'][index]='Operation'
                    for h in Q143_status_action7.index:
                        status =Q143_status_action7['Account Number'][h]
                        if merge_df['Account Number'][index]==status: 
                            merge_df['Risks'][index]='Monitoring and reporting'
                            
merge_df['Population'].fillna(0, inplace = True) 
merge_df = merge_df[merge_df['Population']!=0]

merge_df['City Location'].fillna(0, inplace = True) 
merge_df = merge_df[merge_df['City Location']!=0]
merge_df['longitude'] = merge_df['City Location']
merge_df['latitude'] = merge_df['City Location']
for i in merge_df.index:
    merge_df['longitude'][i] = merge_df['City Location'][i].split()[1].replace('(','')
    merge_df['latitude'][i] = merge_df['City Location'][i].split()[2].replace(')','')

In [ ]:
Fig_Q14_3 = px.scatter_geo(merge_df,lat = 'latitude',lon= 'longitude',size = 'Population',width=1200,height = 1000, color='Risks', hover_name="City",projection="natural earth",color_discrete_map={
                'No investment in existing water supply infrastructure': "#C4181C",
                'Scoping': "#EC4B2E",
                'Pre-feasibility study/impact assessment': "#FA9346",
                'Pre-implementation': "#FBD374",
                'Implementation': "#CBEC6E",
                'Implementation complete but not in operation': "#A1D868",
                'Operation': "#1A7E37",
                'Monitoring and reporting': "#5992BE",
                'Not identified as a risk': "#2E559B"},
                category_orders={'Risks':(
                'No investment in existing water supply infrastructure',
                'Scoping',
                'Pre-feasibility study/impact assessment',
                'Pre-implementation',
                'Implementation',
                'Implementation complete but not in operation',
                'Operation',
                'Monitoring and reporting',
                'Not identified as a risk')
                },  
                hover_data = {'longitude':False,'latitude':False,'Country':True,'Population':True,'CDP Region':True,'Risks':True, 'WPI': True},
                labels = {'Risks':'Status of action'})
Fig_Q14_3.update_layout(
    title={
        'text': "Risk to urban water security",
        'y':0.85,
        'x':0.35,
        'xanchor': 'center',
        'yanchor': 'top'},
    legend_title={
        'text': "Inadequate or ageing water supply infrastructure",
        'side': "top"},
    legend = {
        'yanchor':"top",
        'y':0.9,
        'xanchor':"left",
        'x':1},
    font=dict(
        family="Arial",
        size=12,
        color= "black"),
    autosize=False,
    width=800,
    height=400,
)
Fig_Q14_3 .show()

Some cities take water-related risks well into consideration.
Nevertheless, it seems that many cities underestimate the risks. Both cities with an inadequate water supply system and cities with a well-established but aging system are neglecting these issues.